In [1]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_txt = f.read()
print("Total number of character: ", len(raw_txt))
print(raw_txt[:100])

Total number of character:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [2]:
import re
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [3]:
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [4]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [5]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


根据分词器的example对the-verdict.txt进行分词处理

In [6]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_txt)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
print(preprocessed[:50])

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself']


将分词后的单词按照字母顺序排序

In [7]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


对排好序的、没有重复的单词与字符创建ID

In [8]:
vocab = {token:integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


根据输出可知，词汇表包含了与唯一整数标签相关联的单个token。我们接下来的目标是利用这个词汇表，将新文本转换为token ID

让我们在 Python 中实现一个完整的分词器类，其中包含一个 encode 方法，该方法负责将文本拆分为token，并通过词汇表进行token字符串到整数（token ID）的映射，以通过词汇表生成token ID。此外，我们还将实现一个 decode 方法，该方法则负责进行整数到字符串的反向映射，将token ID 转换回文本。

In [11]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encoder(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decoder(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])

        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)                           
        return text

![SimpleTokenizerV1的图解说明](imgs/PixPin_2025-07-02_21-42-03.png)

In [12]:
tokenizer = SimpleTokenizerV1(vocab=vocab)
text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encoder(text)
print(ids)
print(tokenizer.decoder(ids))

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [13]:
text = "Hello, do you like tea?"
print(tokenizer.encoder(text=text))

KeyError: 'Hello'

这里由于vocab由于没有Hello的映射，因此无法进行encoder，这体现出需要考虑大型和多样化的训练集以扩展词汇的必要性。

因此添加了在词汇表中的单词时使用一个<|unk|> token。此外，我们还会在不相关的文本之间添加一个特殊的<|endoftext|> token。
![Alt text](imgs/PixPin_2025-07-02_22-03-15.png)
![Alt text](imgs/PixPin_2025-07-02_22-03-44.png)

In [25]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab_v2 = {token:integer for integer,token in enumerate(all_tokens)}
print(len(vocab_v2.items()))
for i, item in enumerate(list(vocab_v2.items())[-5:]):
  	print(item)

1132
('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [31]:
class SimpleTokenizerV2:
    def __init__(self,vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encoder(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decoder(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)                    
        return text

In [32]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text_combine = " <|endoftext|> ".join((text1, text2))
print(text_combine)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [33]:
tokenizer = SimpleTokenizerV2(vocab_v2)
ids = tokenizer.encoder(text_combine)
print(ids)

text = tokenizer.decoder(ids=ids)
print(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


**特殊Token**

[BOS]（序列开始）：这个token表示文本的起始位置，指示 LLM 内容的开始。

[EOS]（序列结束）：这个token位于文本的末尾，在连接多个无关文本时特别有用，类似于 <|endoftext|>。例如，在合并两个不同的维基百科文章或书籍时， [EOS] token指示一篇文章结束和下一篇文章开始。

[PAD]（填充）：在使用大于 1 的批量大小数据集训练 LLM 时，批量可能包含不同长度的文本。为了确保所有文本长度一致，较短的文本会用[PAD] token进行扩展或填充，直到达到批量中最长文本的长度。

In [1]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.5.1


In [4]:
tokenizer = tiktoken.get_encoding("gpt2")
#这个tokenizer可以理解为调用tiktoken的gpt2的vocab
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print("ids' length is: ", len(integers))
print("mapping ids are: ", integers)
strings = tokenizer.decode(integers)
print("mapping back to strings: ", strings)

print("\nEach ID and its corresponding token:")
for token_id in integers:
    token_str = tokenizer.decode([token_id])
    print(f"{token_id} -> '{token_str}'")

ids' length is:  20
mapping ids are:  [15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]
mapping back to strings:  Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.

Each ID and its corresponding token:
15496 -> 'Hello'
11 -> ','
466 -> ' do'
345 -> ' you'
588 -> ' like'
8887 -> ' tea'
30 -> '?'
220 -> ' '
50256 -> '<|endoftext|>'
554 -> ' In'
262 -> ' the'
4252 -> ' sun'
18250 -> 'lit'
8812 -> ' terr'
2114 -> 'aces'
286 -> ' of'
617 -> ' some'
34680 -> 'unknown'
27271 -> 'Place'
13 -> '.'


根据输出我们可以看到这里的通过Byte pair encoding，不再是对单词进行mapping，而是通过Byte进行mapping。这样的好处的面对未知词，例如someunknownPlace这一类未曾见过的词，我们可以通过组合vocab中认识的词来表示这个词。
![Alt text](imgs/PixPin_2025-07-03_02-01-53.png)

这里值得注意的是，对于早期的GPT模型的BPE分词器，词汇表大小为50257，<|endoftext|> 被分配了最大的token ID。

In [9]:
#练习2.1
text = "Akwirw ier"
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)
strings = tokenizer.decode(integers)
print(f"raw test is '{text}', remap text is '{strings}'")
print("\nEach ID and its corresponding token:")
for token_id in integers:
    token_str = tokenizer.decode([token_id])
    print(f"{token_id} -> '{token_str}'")

[33901, 86, 343, 86, 220, 959]
raw test is 'Akwirw ier', remap text is 'Akwirw ier'

Each ID and its corresponding token:
33901 -> 'Ak'
86 -> 'w'
343 -> 'ir'
86 -> 'w'
220 -> ' '
959 -> 'ier'


这里借用：[skindhu](https://github.com/skindhu/Build-A-Large-Language-Model-CN/blob/main/cn-Book/2.%E5%A4%84%E7%90%86%E6%96%87%E6%9C%AC%E6%95%B0%E6%8D%AE.md#25-%E5%AD%97%E8%8A%82%E5%AF%B9%E7%BC%96%E7%A0%81byte-pair-encoding)的思考来解释BPE是如何运行的。

> [!TIP]
>
> **个人思考：** 字节对编码是一种基于统计的方法，它会先从整个语料库中找出最常见的字节对（byte pair），然后把这些字节对合并成一个新的单元。让我们用一个具体的示例来描述这个过程：
>
> 假如有句子：“The cat drank the milk because it was hungry”
>
> 1. **初始化：BPE会先将句子中每个字符视为一个单独的token**
>
>    ```
>    ['T', 'h', 'e', ' ', 'c', 'a', 't', ' ', 'd', 'r', 'a', 'n', 'k', ' ', 't', 'h', 'e', ' ', 'm', 'i', 'l', 'k', ' ', 'b', 'e', 'c', 'a', 'u', 's', 'e', ' ', 'i', 't', ' ', 'w', 'a', 's', ' ', 'h', 'u', 'n', 'g', 'r', 'y']
>    ```
>
> 2. **统计最常见的字节对**
>
>    BPE算法会在这些token中找到出现频率最高的“字节对”（即相邻的两个字符），然后将其合并为一个新的token。
>
>    例如这里最常见的字节对时（'t', 'h'），因为它在单词"the"和"that"中出现频率较高。
>
> 3. **合并字节对**
>
>    根据统计结果，我们将最常见的字节对（'t', 'h'）合并为一个新的token，其它类似
>
>    ```
>    ['Th', 'e', ' ', 'c', 'a', 't', ' ', 'dr', 'a', 'nk', ' ', 'th', 'e', ' ', 'm', 'i', 'l', 'k', ' ', 'be', 'c', 'a', 'u', 'se', ' ', 'it', ' ', 'wa', 's', ' ', 'hu', 'n', 'gr', 'y']
>    ```
>
> 4. **重复步骤2和3，得到最终的token序列**
>
>    ```
>    ['The', ' ', 'cat', ' ', 'drank', ' ', 'the', ' ', 'milk', ' ', 'because', ' ', 'it', ' ', 'was', ' ', 'hungry']

2.6 使用滑动窗口进行数据采样
实现一个dataloader，通过滑动窗口方法从训练数据集中提取输入-目标对

In [12]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_txt = f.read()

enc_text = tokenizer.encode(raw_txt)
print(len(enc_text))
print(f"前50个经过BPE的词，{enc_text[:50]}")
#“从数据集中移除前50个token以便演示，因为这会在接下来的步骤中产生稍微更有趣的文本段落。”

5145
前50个经过BPE的词，[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257, 7026, 15632, 438, 2016, 257, 922, 5891, 1576, 438, 568, 340, 373, 645, 1049, 5975, 284, 502, 284, 3285, 326, 11, 287, 262, 6001, 286, 465, 13476, 11, 339, 550, 5710, 465, 12036, 11, 6405, 257, 5527, 27075, 11]


In [16]:
enc_sample = enc_text[50:]
#创建输入-目标对以进行下一个单词预测任务的最简单和最直观的方法之一是创建两个变量x和y，其中x包含输入token，y包含目标，即输入向右移动1位的结果。

context_size = 4 #窗口大小，即可记忆的上下文的长度
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


箭头左侧（---->）的所有内容代表 LLM 将接收到的输入，而箭头右侧的token ID 则表示 LLM 应该预测的目标token ID。

In [21]:
for i in range(1,context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [24]:
#text version
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


![Alt text](imgs/PixPin_2025-07-03_02-36-10.png)

在我们将token转换为嵌入之前，还有一个任务要完成，正如我们在本章开始时提到的：实现一个高效的数据加载器，该加载器遍历输入数据集并将输入和目标作为 PyTorch 张量返回，这些张量可以视为多维数组。

具体来说，我们的目标是返回两个张量：一个输入张量，包括 LLM 看到的文本，另一个目标张量，包含 LLM 需要预测的目标，如图 2.13 所示。

In [25]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids = []     # x
        self.target_ids = []    # y
        token_ids = tokenizer.encode(text) #将原为本的text转换为token id
        
        for i in range(0, len(token_ids) - max_length, stride):     #stride是每次滑动窗口处理 token 的步长，图2.13的步长是1
            input_chunk = token_ids[i: i+ max_length]
            target_chunk = token_ids[i+1: i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk)) #将每行的id作为张量添加到input_ids
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]


> [!NOTE]
> 这里提到了一个变量，stride，是滑动窗口的步长，图2.13的步长为1，stride最小为1，毕竟你得往前走嘛，最大不能超过max_length，因为这样将跳过中间的词。
> | `stride` 值     | 效果        | 说明                    |
> | -------------- | --------- | --------------------- |
> | `1`            | 重叠多，数据最大化 | 提供最多的训练样本，训练更充分，计算也更多 |
> | `= max_length` | 无重叠       | 每段不重叠，效率高但样本数少        |
> | 介于两者之间         | 折中        | 控制样本数量和训练冗余之间的平衡      |


In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(text=txt, tokenizer=tokenizer, max_length=max_length, stride=stride)
    dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=0

    )
    return dataloader